# Custom Ivy Prepackaged Model Server

In this notebook we create a new custom Ivy_Server prepackaged server with two versions:
   * A Seldon protocol Ivy model server
   * A KfServing V2 protocol version using MLServer for running ivy models

The Seldon model server is in defined in `ivyserver` folder.

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:8080`

In [ ]:
!kubectl create namespace seldon

In [20]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [ ]:
VERSION = !cat ../../../version.txt
VERSION = VERSION[0]
VERSION

## Training (can be skipped)

In [1]:
TRAIN_MODEL = True
if TRAIN_MODEL:
    import ivy
    import os

    class Regressor(ivy.Module):
        def __init__(self, input_dim, output_dim, is_training=True):
            self.linear = ivy.Linear(input_dim, output_dim)
            self.dropout = ivy.Dropout(0.5, training=is_training)
            ivy.Module.__init__(self)

        def _forward(self, x, ):
            x = ivy.sigmoid(self.linear(x))
            x = self.dropout(x)
            return x

    ivy.set_backend('torch')  # set backend to PyTorch

    model = Regressor(input_dim=3, output_dim=1)
    optimizer = ivy.Adam(1e-4)

    # generate some random data
    x = ivy.random.random_normal(shape=(100, 3))
    y = ivy.random.random_normal(shape=(100, 1))

    def loss_fn(pred, target):
        return ivy.mean((pred - target)**2)

    for epoch in range(50):
        # forward pass
        pred = model(x)

        # compute loss and gradients
        loss, grads = ivy.execute_with_gradients(lambda v: loss_fn(pred, y), model.v)

        # update parameters
        model.v = optimizer.step(model.v, grads)

        # print current loss
        if (epoch+1) % 10 == 0:
            print(f'Epoch: {epoch + 1:2d} --- Loss: {ivy.to_numpy(loss).item():.5f}')
            
    print('Finished training!')
    
    model_dir = "./artifacts"
    ivy_model = "model.pkl"
    model_file = os.path.join(model_dir, ivy_model)
    
    if not os.path.exists(model_dir): os.makedirs(model_dir)
    model.v.cont_to_disk_as_pickled(model_file)

Epoch: 10 --- Loss: 1.35421
Epoch: 20 --- Loss: 1.46206
Epoch: 30 --- Loss: 1.57076
Epoch: 40 --- Loss: 1.44568
Epoch: 50 --- Loss: 1.62109
Finished training!


In [18]:
%%writefile artifacts/models.py
import ivy

class Regressor(ivy.Module):
    def __init__(self, input_dim, output_dim, is_training=True):
        self.linear = ivy.Linear(input_dim, output_dim)
        self.dropout = ivy.Dropout(0.5, training=is_training)
        ivy.Module.__init__(self)

    def _forward(self, x, ):
        x = ivy.sigmoid(self.linear(x))
        x = self.dropout(x)
        return x
    
input_dim = 3
output_dim = 1
backend = 'torch'

Writing artifacts/models.py


## Update Seldon Core with Custom Model

In [ ]:
%%writetemplate values.yaml
predictor_servers:
  MLFLOW_SERVER:
    protocols:
      seldon:
        defaultImageVersion: "{VERSION}"
        image: seldonio/mlflowserver
  SKLEARN_SERVER:
    protocols:
      seldon:
        defaultImageVersion: "{VERSION}"
        image: seldonio/sklearnserver
      kfserving:
        defaultImageVersion: "0.3.2"
        image: seldonio/mlserver
  TENSORFLOW_SERVER:
    protocols:
      seldon:
        defaultImageVersion: "{VERSION}"
        image: seldonio/tfserving-proxy
      tensorflow: 
        defaultImageVersion: 2.1.0
        image:  tensorflow/serving
  XGBOOST_SERVER:
    protocols:
      seldon:
        defaultImageVersion: "{VERSION}"
        image: seldonio/xgboostserver
      kfserving:
        defaultImageVersion: "0.3.2"
        image: seldonio/mlserver
  IVY_SERVER:
    protocols:
      seldon:
        defaultImageVersion: "{VERSION}"
        image: seldonio/ivyserver
      kfserving:
        defaultImageVersion: "0.3.2"
        image: seldonio/mlserver
  TRITON_SERVER:
    protocols:
      kfserving:
        defaultImageVersion: "21.08-py3"
        image: nvcr.io/nvidia/tritonserver
  TEMPO_SERVER:
    protocols:
      kfserving:
        defaultImageVersion: "0.3.2"
        image: seldonio/mlserver


In [ ]:
!helm upgrade seldon-core  \
    ../../../helm-charts/seldon-core-operator \
    --namespace seldon-system \
    --values values.yaml \
    --set istio.enabled=true

## DeployLightGBM Model with Seldon Protocol

In [ ]:
!cat model_seldon_v1.yaml

Wait for new webhook certificates to be loaded

In [ ]:
import time

time.sleep(60)

In [ ]:
!kubectl create -f model_seldon_v1.yaml -n seldon

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[0].metadata.name}' -n seldon) -n seldon

In [ ]:
for i in range(60):
    state = !kubectl get sdep iris -n seldon -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
import json
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 3.0, 4.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/iris/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

In [ ]:
!kubectl delete -f model_seldon_v1.yaml

## Deploy Model with KFserving Protocol

In [ ]:
!cat model_seldon_v2.yaml

In [ ]:
!kubectl create -f model_seldon_v2.yaml -n seldon

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[0].metadata.name}' -n seldon) -n seldon

In [ ]:
for i in range(60):
    state = !kubectl get sdep iris -n seldon -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
import json
X=!curl -s -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'\
   -X POST http://localhost:8003/seldon/seldon/iris/v2/models/infer \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

In [ ]:
!kubectl delete -f model_seldon_v2.yaml